MDE Dam Safety Weather Notification Code
Version 2.1
March 18, 2024
This version includes:
* Parallelized API calls
* OAuth 2.0 persistence after password changes
* Automated NOAA API calls
* Customizable email preferences
* Renamed variables for clarity

In [27]:
from datetime import datetime
from datetime import timedelta
import os
import subprocess
import nbformat
from nbconvert import HTMLExporter
import IPython.core.display as display

#Put the path to the NOAA + Master Table .xlsx below:
# if it throws a unicode error, replace any \ characters with either \\ or /
# Note, this MUST be the .csv of the Master Table that is outputted by the noaaData script for this code to work.
# The design precipitation column is necessary.
uploaded = 'damsNOAA.csv'

In [28]:
# NOAA data only needs to be updated on rare occasion, so this ensures it is only updated yearly

def is_file_old(filepath):
    if not os.path.exists(filepath):
        return True  # File doesn't exist, so it's considered old
    else:
        file_modified_time = datetime.fromtimestamp(os.path.getmtime(filepath))
        current_time = datetime.now()
        difference = current_time - file_modified_time
        return difference.days > 365  # Check if file is older than 1 year
    
if is_file_old('damsNOAA.csv'):
    print("NOAA data is older than 1 year. Gathering data again...")
    notebook_path = 'noaaData.ipynb'
    # Command to run the Jupyter Notebook file
    command = f'jupyter nbconvert --to notebook --inplace --execute {notebook_path}'

    # Run the command as a subprocess
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Wait for the subprocess to complete
    stdout, stderr = process.communicate()

    # Check the return code to see if the subprocess completed successfully
    if process.returncode == 0:
        print('Notebook executed successfully!')
    else:
        print(f'Error executing notebook: {stderr.decode()}')

13:25:16.574396


In [29]:
# Get the API key from the Windows environment variable
# On MDE systems that prohibit modifying environment variables, run: `rundll32 sysdm.cpl,EditEnvironmentVariables` in cmd to modify
# Environment variables should always be used rather than pasting passwords/keys into code.
apikey = os.environ.get('weatherAPI_key')

In [30]:
## Retry wrapper allows for retrying API pulls/email pushes if there is an error without the code crashing

import time
import pandas as pd

pd.options.display.max_colwidth = 1000

def retryURL(func, retries=10):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            print(f"Running")
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"Error: {e} (attempt {attempts+1}/{retries})")
                
                time.sleep(2)
                attempts += 1

    return retry_wrapper      

@retryURL
def url_to_dataframe(url):
    df = pd.read_json(url)
    return df
          

In [31]:
import asyncio
import aiohttp
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
nest_asyncio.apply()
# EDIT: the __import__ line is not needed, leaving it just 
# so that the comments make sense
# __import__('IPython').embed() 

## Pulls weather data for the coordinates of each dam @retryURL
async def url_to_dataframe(url, session):
    async with session.get(url) as response:
        data = await response.json()
        df = pd.DataFrame(data)
        return df

# set the API POST endpoint as the target URL
weatherURL = "http://api.weatherapi.com/v1/forecast.json"
# This iteration of the code works for the 6-hr storm.
def addPlusSix(row):
    return sum(row[0:6])

# if the dam data is in excel format:
#dams = pd.read_excel(uploaded)
# if the dam data is in csv format:
dams = pd.read_csv(uploaded)

async def process_dam(dam_idx, session):
    df = await url_to_dataframe(f"{weatherURL}?key={apikey}&q={dams['Latitude'][dam_idx]},{dams['Longitude'][dam_idx]}&days=4&aqi=no&alerts=no", session)
    blankData = []
    fhr = pd.DataFrame(blankData)

    subsequentSixMD = [None] * 96

    # Next, pull the hourly data out of the .json
    for item in df['forecast']['forecastday']:
        fhr = pd.concat([fhr, pd.DataFrame(item['hour'])])

    # Add the forecasted rainfall amount for the next 96 hrs
    for hours in range(0, 96):
        subsequentSixMD[hours] = round(addPlusSix(fhr['precip_in'][hours:hours+6].tolist()),3)
   

    return subsequentSixMD
forcast_date = None
async def main():

    async with aiohttp.ClientSession() as session:
        tasks = [asyncio.create_task(process_dam(dam_idx, session)) for dam_idx in range(len(dams))]
        results = await asyncio.gather(*tasks)

    # Collect the results
    serMD = [result for result in results]

    # Assign the results to the dams DataFrame
    dams['Sub6MD'] = serMD

    print(dams['Sub6MD'])

# Run the main function in a separate thread
executor = ThreadPoolExecutor(max_workers=1)
loop = asyncio.get_event_loop()
future = executor.submit(loop.run_until_complete, main())
future.result()



0                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1                                 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2                           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [33]:
## Filter so that only the first hourly occurence of rain per dam is maintained
def filter_unique_first_values(input_list):
    unique_first_values = {}
    result = []

    for sublist in input_list:
        first_value = sublist[0]
        if first_value not in unique_first_values:
            unique_first_values[first_value] = True
            result.append(sublist)

    return result

In [34]:
## Compare the Sub6 rainfall to the design storm data
# Returns the State ID of a dam whose forcasted rain greater than its
# design storm and the hour at which that happens for the four days after the script is run.

def designExceeded(dams):

  highRainDam = []

  for dam in range(len(dams)):
    for hour in range(len(dams['Sub6MD'][dam])):
      # For testing, see if it exceeds 0 to check for any rain. Exceedance of designPrecip is usually None.
      if dams['Sub6MD'][dam][hour] > dams['designPrecip'][dam]:
        highRainDam.append([(dams.loc[dam]['State ID']), hour])


  return highRainDam

In [35]:
print(type((dams['Sub6MD'][1])))

<class 'list'>


In [36]:
exceededDams = designExceeded(dams)

In [37]:
## Format time to make it more readable in the email body.
from datetime import datetime, timedelta

def formatTime(hour):
  d = datetime.strptime(str(datetime.now().date()), "%Y-%m-%d")

  dOffset = timedelta(hours=hour)
  new_time = (d + dOffset).strftime("%I:%M %p" " on %a, %b %d")
  return new_time

In [38]:
print(formatTime(3))

03:00 AM on Tue, Mar 19


In [39]:
## Filter so that only the first hourly occurence of rain per dam per day is maintained
def find_first_unique_per_id_blocks(input_list):
    # Create a dictionary to store first unique entries per ID
    first_unique_entries = {}

    # Iterate through the input list
    for entry in input_list:
        current_id, current_hour = entry

        # Calculate the 24-hour block
        block = current_hour // 24

        # Generate a unique identifier for the entry
        entry_identifier = (current_id, block)

        # Check if the entry identifier is in the dictionary
        if entry_identifier not in first_unique_entries:
            first_unique_entries[entry_identifier] = entry
        else:
            stored_hour = first_unique_entries[entry_identifier][1]
            if current_hour < stored_hour:
                first_unique_entries[entry_identifier] = entry

    return list(first_unique_entries.values())

In [40]:
exceededDams4Days = find_first_unique_per_id_blocks(exceededDams)

In [41]:
# RAIN!! Add more rain to the results for testing purposes.

#exceededDams4Days.append([8,5])
#exceededDams4Days.append([12,92])
#print((exceededDams4Days))

In [42]:
## Form the body of the email notification

notif_body = []
notif_ID = []
notifListWeekend_header = []
notifListWeekend_footer = []
body_df = pd.DataFrame(columns=['msgBody','Dam ID','Inspector Initials', 'County'])

# Check to see if any dams will experience rainfall in excess of their design storm.
if exceededDams4Days != []:
  
  # if so, add a warning message (with hmtl formatting)
  notifListWeekend_header.append(f"Weather forecasts indicate heavy rainfall at the following dam(s) exceeding the region's 100-year storm.")
  notifListWeekend_header.append('\n')
  notifListWeekend_header.append(f"Note, this is not an indication that a dam is near failure; rather, it indicates that increased surveilance is necessary.")
  notifListWeekend_header.append('\n' '<pre>' '\n' ) # HTML tag <pre> to preserve whitespace

  for entry in exceededDams4Days:
    block = entry[1] // 24
    damIndex = entry[0]-1

    notif_body.append(f'''   MD Dam No. {dams['State ID'][damIndex]}: {dams['Dam Name'][damIndex]}:''')
    notif_body.append('\n')
    notif_body.append(f"     Rainfall totaling {format(max((dams['Sub6MD'][damIndex])[block*24:(block+1)*24]), '.2f')} in. starting at ")
    notif_body.append(formatTime(entry[1]))
    notif_body.append('\n' f"     100-year rainfall {format(dams['designPrecip'][damIndex], '.2f')} in.")
    notif_body.append('\n' '\n')

    bodyData = {'msgBody' : notif_body, 'Dam ID' : dams['State ID'][damIndex], 'Inspector Initials' : dams['Inspector'][damIndex], 'County' : dams['County'][damIndex]}
    body_df.loc[len(body_df)] = pd.Series(bodyData).transpose()
    print(body_df)

  
  notifListWeekend_footer.append('</pre>') # Close HTML, return to normal text
  notifListWeekend_footer.append('\n' 'This is an automated email. For more information, please reach out to Ariel Ben-Sorek of the Dam Safety Permits Division at ariel.ben-sorek@maryland.gov')

else:
  notifListWeekend_header.append(f'''No high-rainfall events are predicted in the next 96 hours.''')  

In [43]:
def series_to_condensed_string(Series):
    # Weather alerts in body_df are in a Series, which messes with their formatting.
    # Convert them to a string nicely with this function. Check first for None to prevent errors.

    if Series.empty:
        return
    else:
        my_list = Series.to_list()
        my_list = ''.join(my_list[0])
        return my_list


In [44]:
## Converts strings to html text for email bodies

def format_string_as_html(input_string):

    if input_string == None:
        return
    else:
        # Replace newline characters with <br> tags
        html_content = input_string.replace('\n', '<br>')

        # Wrap the content in HTML tags
        formatted_html = f'''<html><body>{html_content}</body></html>'''

        return formatted_html

In [45]:
## Composes the email message by joining the proper header, body, and footer

def composeMessage(notifListWeekend_header, notif, notifListWeekend_footer):
    header_str = ''.join(notifListWeekend_header)
    body_str = ''.join(notif)
    footer_str = ''.join(notifListWeekend_footer)
    
    msg = header_str + body_str + footer_str
    return(msg)

In [46]:
## Compose email list, called 'notif_by_inspector'

# Add inspectors emails by initials in the Master Table
# If someone has no dams, add them in manually
notif_by_inspector = pd.DataFrame()
damlessInitials = ['JTR']
inspectorInitials = ((dams['Inspector'].dropna()).unique()).tolist()

allInitials = inspectorInitials + damlessInitials
notif_by_inspector['Initials'] = allInitials
print(allInitials)

# Email mapping list
# for testing, use this email list. Otherwise, use peoples' real adresses.

#email_mapping = [('ABS', 'ariel.ben-sorek@maryland.gov'), ('ASR', 'ariel.ben-sorek@maryland.gov'),('CHW', 'ariel.ben-sorek@maryland.gov'), ('HS', 'ariel.ben-sorek@maryland.gov'), ('JTR', 'ariel.ben-sorek@maryland.gov'), ('KRF', 'ariel.ben-sorek@maryland.gov'), ('RCS', 'ariel.ben-sorek@maryland.gov'), ('SEB', 'ariel.ben-sorek@maryland.gov'), ('VPD', 'ariel.ben-sorek@maryland.gov'), ('WSA', 'ariel.ben-sorek@maryland.gov')]

email_mapping = [('ABS', 'ariel.ben-sorek@maryland.gov'), ('ASR', 'ariel.ben-sorek+ASR@maryland.gov'),('CHW', 'ariel.ben-sorek+CHW@maryland.gov'), ('HS', 'ariel.ben-sorek+HS@maryland.gov'), ('JTR', 'ariel.ben-sorek+JTR@maryland.gov'), ('KRF', 'ariel.ben-sorek+KRF@maryland.gov'), ('RCS', 'ariel.ben-sorek+RCS@maryland.gov'), ('SEB', 'ariel.ben-sorek+SEB@maryland.gov'), ('VPD', 'ariel.ben-sorek+VPD@maryland.gov'), ('WSA', 'ariel.ben-sorek+WSA@maryland.gov')]

#email_mapping = [('ABS', 'ariel.ben-sorek@maryland.gov'), ('ASR', 'anna.sobilo-ryzner@maryland.gov'),('CHW', 'charles.wallis@maryland.gov'), ('HS', 'hira.shrestha@maryland.gov'), ('JTR', 'john.roche@maryland.gov'), ('KRF', 'kelly.flint@maryland.gov'), ('RCS', 'roxanne.sanderson@maryland.gov'), ('SEB', 'scott.bass@maryland.gov'), ('VPD', 'visty.dalal@maryland.gov'), ('WSA', 'william.ashby@maryland.gov')]

# Opt-in list: No emails; Own dams; all dams; or all dams plus system notifications
email_optin = [('ABS', 'system'), ('ASR', 'no'),('CHW', 'all'), ('HS', 'own'), ('JTR', 'all'), ('KRF', 'own'), ('RCS', 'own'), ('SEB', 'no'), ('VPD', 'no'), ('WSA', 'custom_WSA')]

# Create a dictionary from the email_mapping list
email_dict = dict(email_mapping)
email_optin_dict = dict(email_optin)

# Create a new column 'Email' using .map() based on the 'Initials' column
notif_by_inspector['Email'] = notif_by_inspector['Initials'].map(email_dict)
notif_by_inspector['Opt-in'] = notif_by_inspector['Initials'].map(email_optin_dict)
print(notif_by_inspector)

['ASR', 'WSA', 'ABS', 'SEB', 'HS', 'KRF', 'VPD', 'RCS', 'CHW', 'JTR']
  Initials                             Email      Opt-in
0      ASR  ariel.ben-sorek+ASR@maryland.gov          no
1      WSA  ariel.ben-sorek+WSA@maryland.gov  custom_WSA
2      ABS      ariel.ben-sorek@maryland.gov      system
3      SEB  ariel.ben-sorek+SEB@maryland.gov          no
4       HS   ariel.ben-sorek+HS@maryland.gov         own
5      KRF  ariel.ben-sorek+KRF@maryland.gov         own
6      VPD  ariel.ben-sorek+VPD@maryland.gov          no
7      RCS  ariel.ben-sorek+RCS@maryland.gov         own
8      CHW  ariel.ben-sorek+CHW@maryland.gov         all
9      JTR  ariel.ben-sorek+JTR@maryland.gov         all


In [47]:
## Determine who wants what kind of email, and compose that email.

def compositionCaller(initials):

    # Check first if the initials are valid to avoid crashing
    opt_series = notif_by_inspector[notif_by_inspector['Initials'] == initials]['Opt-in']
    if opt_series.empty:
        print(f"No such inspector {initials}")
        return
    
    opt_status = notif_by_inspector[notif_by_inspector['Initials'] == initials]['Opt-in'].iloc[0]

    if opt_status == 'no':
        print(f"Inspector {initials } opt-in status is 'no'")
        return # Send no email if someone has not opted in
        
    if opt_status == 'own':
        print(f"Inspector {initials } opt-in status is 'own'")
        if (body_df[body_df['Inspector Initials'] == initials]['msgBody']).empty:
            return # Send no email if none of their own dams have exceedences
        else:
            formatted_body = series_to_condensed_string(body_df[body_df['Inspector Initials'] == initials]['msgBody'])
            return composeMessage(notifListWeekend_header, formatted_body, notifListWeekend_footer) # Send if their own dam has exceedences
        
    if opt_status == 'custom_WSA':
        print(f"Inspector {initials } opt-in status is '{opt_status}'")
        # Bill asked to be notified about his own dams, OR any dams in Western MD counties
        custom_WSA_bodies = body_df[(body_df['Inspector Initials'] == initials) | (body_df['County'] == 'Garrett') | (body_df['County'] == 'Allegany')]['msgBody']
        if custom_WSA_bodies.empty:
            return # Send no email if none of their own dams have exceedences
        else:
            formatted_body = series_to_condensed_string(custom_WSA_bodies)
            return composeMessage(notifListWeekend_header, formatted_body, notifListWeekend_footer) # Send if their own dam has exceedences
    

    if opt_status == 'all':
        print(f"Inspector {initials } opt-in status is '{opt_status}'")
        if (body_df['msgBody']).empty:
            return # Send no email if no dams have exceedences
        else:
            formatted_body = series_to_condensed_string(body_df['msgBody'])
            return composeMessage(notifListWeekend_header, formatted_body, notifListWeekend_footer) # Send if any dams have exceedences  

    if opt_status == 'system':
        print(f"Inspector {initials } opt-in status is {opt_status}")
        if (body_df['msgBody']).empty:
            print('System user: No high rainfall events.')
            return composeMessage(notifListWeekend_header, '', notifListWeekend_footer) # Send 'no exceedences' email
        else:
            print('System user: high rainfall event(s)')
            formatted_body = series_to_condensed_string(body_df['msgBody'])
            return composeMessage(notifListWeekend_header, formatted_body, notifListWeekend_footer) # Send if any dams have exceedences
    

In [48]:
## Send the email using OAuth

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import base64
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.exceptions import RefreshError



SCOPES = ['https://mail.google.com/']
creds = None

# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        try:
            creds.refresh(Request())
        except RefreshError as e:
            # Revoked credentials, handle the error and get new credentials
            print("Credentials revoked, likely due to password reset.")
            os.remove('token.json')
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

def gmail_send_message(html_content, receiver):

    try:
        service = build('gmail', 'v1', credentials=creds)
        message = MIMEMultipart("alternative")


        message['To'] = receiver
        message['From'] = 'ariel.ben-sorek@maryland.gov'
        
        if exceededDams4Days != []:
            message['Subject'] = "Maryland Dam Safety Weather Alert"
        
        else:
            message['Subject'] = "Maryland Dam Safety: No severe weather predicted"


        html_part = MIMEText(html_content, "html")
        message.attach(html_part)  

        # encoded message
        encoded_message = base64.urlsafe_b64encode(message.as_bytes()) \
            .decode()

        create_message = {
            'raw': encoded_message
        }
        # pylint: disable=E1101
        send_message = (service.users().messages().send
                        (userId="me", body=create_message).execute())
        print(F'Message Id: {send_message["id"]}')
    except HttpError as error:
        print(F'An error occurred: {error}')
        send_message = None
    return send_message 
      
    

In [49]:
## Send emails to one inspector
@retryURL
def email_one_inspector(initials):
    text = compositionCaller(initials)
    email_date = datetime.now().strftime("%B %d, %Y")
    inspector_opt = notif_by_inspector[notif_by_inspector['Initials'] == initials]['Opt-in'].iloc[0]
    inspector_email = (notif_by_inspector[notif_by_inspector['Initials'] == initials]['Email']).iloc[0]

    if inspector_opt == 'none':
        print(f'{initials} has not opted-in. No email will be sent.')
    
    elif text == None and inspector_opt != 'system':
        print(f'No exceedences at dams for inspector {initials}. No email will be sent.')
  
    else:

        html = f"""\
        <p> <img src="https://mde.maryland.gov/Style%20Library/egov/img/agencyLogo-generic.png" alt = "Maryland Department of the Environment"  style="width:62px;height:64px;margin-right: 15px;float: left;">

        <h1 style="line-height:1.2;font-family:helvetica;font-size:25px;margin:0px 0px 0px;">Maryland Dam Safety Weather Alert</h1>
        </p>

        <h2 style="line-height:0.1;font-family:helvetica;font-size:15px;margin:0px 0px 0px;">{email_date}</h2>
        </p>
        <br>
        <p style="line-height:1.4;font-family:helvetica;">{format_string_as_html(text)}</p>
        """

        gmail_send_message(html, inspector_email)

In [50]:
## Email everyone by looping `email_one_inspector` over all inspectors

def email_Everyone():
    for entry in notif_by_inspector['Initials']:
        print(entry)
        email_one_inspector(entry)

In [51]:
email_Everyone()

ASR
Running
Inspector ASR opt-in status is 'no'
No exceedences at dams for inspector ASR. No email will be sent.
WSA
Running
Inspector WSA opt-in status is 'custom_WSA'
No exceedences at dams for inspector WSA. No email will be sent.
ABS
Running
Inspector ABS opt-in status is system
System user: No high rainfall events.


Message Id: 18e5721f42857ccf
SEB
Running
Inspector SEB opt-in status is 'no'
No exceedences at dams for inspector SEB. No email will be sent.
HS
Running
Inspector HS opt-in status is 'own'
No exceedences at dams for inspector HS. No email will be sent.
KRF
Running
Inspector KRF opt-in status is 'own'
No exceedences at dams for inspector KRF. No email will be sent.
VPD
Running
Inspector VPD opt-in status is 'no'
No exceedences at dams for inspector VPD. No email will be sent.
RCS
Running
Inspector RCS opt-in status is 'own'
No exceedences at dams for inspector RCS. No email will be sent.
CHW
Running
Inspector CHW opt-in status is 'all'
No exceedences at dams for inspector CHW. No email will be sent.
JTR
Running
Inspector JTR opt-in status is 'all'
No exceedences at dams for inspector JTR. No email will be sent.


In [52]:
## Counting rainfall above threshold

#threshold = 0.03 #inches of rain
#def has_over_threshold(lst):
#    return any(x >= threshold for x in lst)

#over_threshold_count = dams['Sub6MD'].apply(lambda x: sum(1 for val in x if val >= threshold)).sum()
#total_entries = dams['Sub6MD'].apply(len).sum()
#print("Number of entries with nonzero values:", over_threshold_count, "or ", over_threshold_count/total_entries*100, " percent.")

#over_threshold_lists = dams[dams['Sub6MD'].apply(has_over_threshold)]
#display(over_threshold_lists)

